In [1]:
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map 
import numpy as np
import random
from prettytable import PrettyTable

dice = [1, 2, 3, 4, 5, 6]   #my dice

ID_dict={"x_00":1,"x_20":2,"x_22":3,"x_32":4,"x_30":5,"x_40":6,"x_50":7} #ID all the states with 1-7

def firstNsecondMax(x):
    "Functions takes a list x, counts number of 1,2,3,4,5,6's"
    "returns first most common number and second most common number in a list x, in a list of len()=2"
    
    l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
    A = np.argmax(l)+1                                                      
    new_l=l
    new_l[np.argmax(new_l)]=0
    A1 = np.argmax(new_l)+1
    return [A,A1]

def State(l):
    "Function takes list l"
    "return state of a roll"
    if max(l)==5:                #check if its AAAAA
        state="x_50"
    elif max(l)==4:              #check if its AAAAB
        state="x_40"
    elif max(l)==3:              #check if its AAABB or AAABC
        new_l= l
        # removing the largest element from temp list
        new_l.remove(max(new_l))
        if max(new_l)==2:        #check if its AAABB  
            state="x_32"
        else:                    #else its AAABC   
            state="x_30"
    elif max(l)==2:
        new_l= l
        # removing the largest element from temp list
        new_l.remove(max(new_l)) 
        if max(new_l)==2:        #check if its AABBC
            state="x_22"
        else:                    #else its AABCD
            state="x_20"
    
    
    else:                        #else its ABCDE
        state="x_00"
    return state    
  
def Re_Roll(state,A,A1):
        "Function takes state of a list (state), the most appearing number (A), second most appearing number (A1) " 
        "returns a list containing new state and and the same A" 
        if state=="x_50":
          x=[A,A,A]
          x_ = random.choices(dice,k=2)
          x.append(x_[0])
          x.append(x_[1])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]
        elif state=="x_40": 
          x=[A,A,A,A1]
          x_ = random.choices(dice,k=1)
          x.append(x_[0])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]   
        elif state=="x_30":
          x=[A,A,A,A1]
          x_ = random.choices(dice,k=1)
          x.append(x_[0])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]
        elif state=="x_32":
          x=[A,A,A,A1,A1]
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]
        elif state=="x_22":
          x=[A,A,A1,A1]
          x_ = random.choices(dice,k=1)
          x.append(x_[0])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]
        elif state=="x_20":
          x=[A,A,A1]
          x_ = random.choices(dice,k=2)
          x.append(x_[0])
          x.append(x_[1])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]
        else:
          x= random.choices(dice,k=5)
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
          A=firstNsecondMax(x)[0]
          A1=firstNsecondMax(x)[1]
        return [state, A, A1] 
for roll in range(3):          
  def results(j):
    
      "Returns the state ID of of the list i.e. x_00, x_20, x_22, x_32, x_30, x_40, x_50"
      x = random.choices(dice,k=5)
      #count the number of 1s 2s 3s 4s 5s 6s and store in a list l
      l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
      state=State(l)
      A=firstNsecondMax(x)[0]
      A1=firstNsecondMax(x)[1]
      for r in range(roll):
        R=Re_Roll(state, A, A1)
        state=R[0]
        A=R[1]
        A1=R[2]
      return ID_dict[state]      #return the state ID


  reps = int(1e6) # number of repetions of game
  cores = 4 # number of cores to be used in parallel 

  # run the function results reps times (chunksize is how many executions are clustered together for one run on a single core)
  res = process_map( results, range(reps), max_workers=cores, total=reps, chunksize=int(reps/1000), tqdm_class=tqdm)


  # print how often we got which state
  print("Roll number ",roll+1," probabilities.")
  print('[x_00, x_20, x_22, x_32, x_30, x_40, x_50]=', np.bincount(res)[1:] / reps, "respectively." )
  print("____________________________________________________________________________________________")


100%|██████████| 1000000/1000000 [00:42<00:00, 23383.22it/s]

Roll number  1  probabilities.
[x_00, x_20, x_22, x_32, x_30, x_40, x_50]= [0.092342 0.462681 0.231752 0.038598 0.154383 0.019443 0.000801] respectively.
____________________________________________________________________________________________



100%|██████████| 1000000/1000000 [01:22<00:00, 12149.93it/s]


Roll number  2  probabilities.
[x_00, x_20, x_22, x_32, x_30, x_40, x_50]= [8.60500e-03 1.97726e-01 3.30342e-01 1.86319e-01 2.33241e-01 4.36770e-02
 9.00000e-05] respectively.
____________________________________________________________________________________________


100%|██████████| 1000000/1000000 [02:05<00:00, 7936.74it/s]


Roll number  3  probabilities.
[x_00, x_20, x_22, x_32, x_30, x_40, x_50]= [7.67000e-04 6.97620e-02 2.87562e-01 3.60451e-01 2.29712e-01 5.17370e-02
 9.00000e-06] respectively.
____________________________________________________________________________________________
